### Imports

In [1]:
import numpy as np
from specmf.models import Graph, MultiFidelityModel
from specmf.utils import error_analysis, val_test_split
from specmf.data import load_data
from specmf.plot import *

In [ ]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 14px;
</style>

### Data loading

In [ ]:
dataset_names = [
    "darcy-flow", 
    "inclusion-field", 
    "inclusion-qoi",
    "beam",
    "cavity",
    "burgers",
]

dataset_name = dataset_names[0]

x_LF, x_HF = load_data(
    dataset_name,
    preprocess=True,
    normalize=True, 
    flatten=True,
    )

print(f"{x_LF.shape=}", f"{x_HF.shape=}")

### Plot data 

In [ ]:
plot_data(x_LF, x_HF, dataset_name)

### Multi-fidelity model

#### Create graph and model instances

In [ ]:
# Create the graph
graph_config = {
    'metric': 'euclidean',
    'dist_space': 'ambient',
    'n_components': None,
    'method': 'full',
    'k_nn': None,
    'corr_scale': None,
    'k_adj': 7,
    'p': 0.5,
    'q': 0.5,
}
g_LF = Graph(data=x_LF, **graph_config)

# Create the model 
model_config = {
    'sigma': 0.01,
    'method': 'full'
}
model = MultiFidelityModel(**model_config)

#### Perform spectral clustering

In [ ]:
n_HF = 100
inds_train, labels = model.cluster(g_LF, n_HF)

In [ ]:
plot_cluster_size_hist(labels)

In [ ]:
# Plot Laplacian spectrum
eigvals, eigvecs = g_LF.laplacian_eig()
plot_spectrum(eigvals, 50)

#### Split high-fidelity data

In [ ]:
inds_val, inds_test = val_test_split(
    n_data=x_HF.shape[0], 
    inds_train=inds_train, 
    split_ratio=0.5,
)

x_HF_train = x_HF[inds_train, :]
x_HF_val = x_HF[inds_val, :]
x_HF_test = x_HF[inds_test, :]

print(f"{x_HF_train.shape=}", f"{x_HF_val.shape=}", f"{x_HF_test.shape=}")

#### Hyper-parameters search

In [ ]:
# from skopt import gp_minimize
# from skopt.space import Integer, Real
# from skopt.utils import use_named_args

# dimensions = [
#     Integer(name='beta', low=1, high=8,),
#     Real(name='kappa', low=1e-8, high=1e3, prior='log-uniform')
#     ]

# @use_named_args(dimensions=dimensions)
# def cost_fn(beta, kappa):
#     """
#     Custom cost function for the optimization.
#     """

#     model.beta = beta
#     model.kappa = kappa

#     x_MF, _, _ = model.transform(g_LF, x_HF_train, inds_train)
#     x_LF = g_LF.nodes
#     _, e_MF = error_analysis(
#         x_LF[inds_val],
#         x_MF[inds_val],
#         x_HF_val,
#         return_values=True,
#         verbose=0,
#     )
#     return np.mean(e_MF)

# res = gp_minimize(
#     func=cost_fn,  
#     dimensions=dimensions,      
#     acq_func="EI",
#     n_calls=200,
#     n_initial_points=50,  
#     random_state=42,
#     verbose=1,
#     initial_point_generator='lhs',
# )

# best_config = {
#     'beta': res.x[0],
#     'kappa': res.x[1],
# }

# print(f"\nBest configuration: \n"
#       f"  - beta = {best_config['beta']}, \n"
#       f"  - kappa = {best_config['kappa']}, \n")
# print(f"  - cost = {res.fun}")

# x = np.array(res.x_iters)[:, 1]
# y = np.array(res.x_iters)[:, 0]
# c = res.func_vals

# fig, ax = plt.subplots(1, 1, figsize=(12, 8))
# ax1 = ax.scatter(x, y, c=c,)
# ax.set_xscale('log')
# ax.set_ylabel(r'$\beta$')
# ax.set_xlabel(r'$\kappa$')
# plt.colorbar(ax1, label='Validation Error')

In [ ]:
def cost_fn(kappa):
    """
    Custom cost function for the optimization.
    """
    # Create the model 
    model_config = {
        'sigma': 0.01,
        'method': 'trunc',
        'spectrum_cutoff': 1200,
    }
    model = MultiFidelityModel(**model_config)

    model.kappa = kappa

    x_MF, _, dPhi = model.transform(g_LF, x_HF_train, inds_train)
    x_LF = g_LF.nodes
    _, e_MF = error_analysis(
        x_LF[inds_val],
        x_MF[inds_val],
        x_HF_val,
        return_values=True,
        verbose=0,
    )
    return np.mean(e_MF), np.mean(dPhi)

Kappas = np.logspace(-12, 8, 50)

errors = []
dPhis = []

for kappa in Kappas:
    print(f"Evaluating kappa = {kappa}")
    error, dPhi = cost_fn(kappa=kappa)
    errors.append(error)
    dPhis.append(dPhi)


In [ ]:
ind_sigma = np.argmin(np.abs(np.array(dPhis) - 3 * model_config["sigma"]))
kappa_sigma = Kappas[ind_sigma]
print(f"Optimal kappa = {kappa_sigma}")

In [ ]:
# Create the figure and the first axis
fig, ax1 = plt.subplots(figsize=(8, 5))

# Plot the first set of data
ax1.plot(Kappas, errors , 'b-')
ax1.set_xscale('log')
ax1.set_xlabel(r'$\kappa$', fontsize=14)
ax1.set_ylabel('error', color='b', fontsize=14)
ax1.tick_params(axis='y', labelcolor='b')

# Create a twin axis sharing the same x-axis
ax2 = ax1.twinx()

# Plot the second set of data
ax2.plot(Kappas, dPhis, 'r-')
ax2.set_yscale('log')
ax2.set_ylabel(r'$\mathrm{mean}(\sqrt{C_{ii}})$', color='r', rotation=0, labelpad=50, fontsize=14)
ax2.tick_params(axis='y', labelcolor='r')

ax1.plot([kappa_sigma, kappa_sigma], [3.5, 19.5], 'k--')
ax2.grid('on')

#### Train the model with optimized hyperprameters

In [ ]:
best_config = {
    'kappa': kappa_sigma,
}

model_config.update(best_config)
model = MultiFidelityModel(**model_config)

In [ ]:
x_MF, C_phi, dPhi = model.transform(g_LF, x_HF_train, inds_train)
model.summary()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.hist(dPhi, bins=20)
ax.set_xlabel("Variance", fontsize=14)
ax.set_ylabel("Frequency", fontsize=14)
ax.grid(True, linestyle="--", linewidth=0.5)
plt.tight_layout()
ax.tick_params(axis="both", labelsize=12)
ax.set_title("Variance histogram", fontsize=18)

### Results

In [ ]:
# Error Analysis for validation datadet
error_analysis(x_LF[inds_val], x_MF[inds_val], x_HF_val)

In [ ]:
# Error Analysis for unseen test datadet
error_analysis(x_LF[inds_test], x_MF[inds_test], x_HF_test)

In [ ]:
# Error Analysis for the whole dataset
error_analysis(x_LF, x_MF, x_HF)

In [ ]:
E_LF = 100 * np.linalg.norm(x_LF - x_HF, axis=1) / (np.mean(np.linalg.norm(x_HF, axis=1)) + 1e-3)
E_MF = 100 * np.linalg.norm(x_MF - x_HF, axis=1) / (np.mean(np.linalg.norm(x_HF, axis=1)) + 1e-3)

plot_distributions(E_LF, E_MF, bins_LF=50, bins_MF=50, mask=None)

In [ ]:
plot_mf_comparison(x_LF, x_MF, x_HF, dataset_name, dPhi=dPhi, inds_centroids=inds_train)